# Appendices

Source code to replicate key results from the appendix

In [4]:
# Libraries and imports
import sys
sys.path.append('../')

import os
import numpy as np
import matplotlib.pyplot as plt
from utils import helpers
import torchvision
from PIL import Image
from torchvision.models import resnet50
import timeit
import torch
from spectral_sobol.torch_explainer import WaveletSobol


## Number of designs

The number of designs corresponds to the number of deviations around the mean sequence. It is used to compute the variance of the Sobol indices. However, a high number of designs leads to an increased computational burden. 

We study the effect of the number of designs on the estimation of the Sobol indices

In [7]:
# Set up 

device = 'cuda:1'
model = resnet50(pretrained = True).to(device)

classes = {
 'fox.png': 278,
 'snow_fox.png': 279,
 'polar_bear.png': 296,
 'leopard.png': 288,
 'fox1.jpg': 277,
 'fox2.jpg': 277,
 'sea_turtle.jpg': 33,
 'lynx.jpg': 287,
 'cat.jpg': 281,
 'otter.jpg': 360
}

image_names = list(classes.keys())

images = [Image.open('../assets/{}'.format(img_name)) for img_name in image_names]

# transforms
normalize = torchvision.transforms.Normalize(mean=[0.485, 0.456, 0.406],
                                 std=[0.229, 0.224, 0.225])

preprocessing = torchvision.transforms.Compose([
    torchvision.transforms.Resize(256),
    torchvision.transforms.CenterCrop(224),
    torchvision.transforms.ToTensor(),
    normalize,
])
 
x = torch.stack(
    [preprocessing(img) for img in images]
).to(device)

y = np.array([classes[img_name] for img_name in image_names])

# loop with a varying number of designs

variants = []
times = []

for i in range(5):

    print('Testing with i = {}'.format(i))
    wavelet = WaveletSobol(model, grid_size = 28, nb_design = 2 ** (i+1), batch_size = 128)

    start = timeit.default_timer()
    variants.append(wavelet(x,y))
    stop = timeit.default_timer()

    times.append(stop - start)

Testing with i = 0


RuntimeError: CUDA out of memory. Tried to allocate 196.00 MiB (GPU 1; 11.90 GiB total capacity; 6.06 GiB already allocated; 11.62 MiB free; 6.07 GiB reserved in total by PyTorch) If reserved memory is >> allocated memory try setting max_split_size_mb to avoid fragmentation.  See documentation for Memory Management and PYTORCH_CUDA_ALLOC_CONF

In [ ]:
# plots

size = 224 # input size
levels = 3 # number of levels in the wavelet decomposition

n_rows, n_cols = len(images), len(variants)

fig, ax = plt.subplots(n_rows, n_cols, figsize = (4 * n_rows, 4 * n_cols))

for j in range(n_cols):

    variant = variants[i] # retrieve the list of wcams
    
    for k in range(n_rows):
        
        ax[k,j].set_title('Nb design : {}'.format(2 ** (j+1)))
        ax[k,j].imshow(variant[k])
        helpers.add_lines(size, levels, ax[k,j])

fig.tight_layout()
plt.axis('off')


plt.savefig('../figs/nb_design.pdf')
plt.show()




## Sampler and estimators

We now consider different samplers and estimators than the Jansen estimator and the ScipySobolSequence. See the notebook `samplers-and-estimators.ipynb` for the computations.